## Project Title: Analyzing Amazon Reviews for Product Insights
### The P < 0.05 Team - Haig Bedros, Noori Selina, Julia Ferris, Matthew Roland

## Load the data

In [1]:
import pandas as pd

url = 'https://media.githubusercontent.com/media/hbedros/data620-final-project/main/electronics.csv'
df = pd.read_csv(url,index_col=0)

### The Question:
How do different product features and review characteristics affect customer satisfaction as shown by review ratings on Amazon?


### Data Sources:
We will use the Amazon Review Data (2018) provided by Jianmo Ni from UCSD. This dataset includes reviews (ratings, text, helpfulness votes), product details (descriptions, categories, price, brand, and image features), and links (also viewed/also bought graphs). It covers reviews from May 1996 to October 2018 and includes 233.1 million reviews. 

The dataset can be found [here](https://nijianmo.github.io/amazon/index.html).


**1. Data Preparation and Cleaning**
- **Assigned to:** Matt and Haig
- **Responsibilities:**
  - Load the dataset and parse the JSON files.
  - Clean the data by removing HTML/CSS content, duplicates, and irrelevant information.

The large dataset was cut down to a smaller size. The process is shown in the file Code_to_Filter_Electronigs_Dataset.ipynb

The data left in the final csv file named electronics.csv is from after 2016.



**2. Descriptive Analysis and Visualization**
- **Assigned to:** Julia
- **Responsibilities:**
  - Perform basic statistics on the dataset (e.g., average rating, number of reviews per product).
  - Create visualizations to show key metrics like rating distribution, number of reviews per category, and product pricing.


**3. Network Analysis**
- **Assigned to:** Selina
- **Responsibilities:**
  - Create and analyze the "also viewed" and "also bought" graphs to find popular product groups.
  - Use centrality measures to find important products in these networks.


**4. NLP and Text Mining**
- **Assigned to:** Matt
- **Responsibilities:**
  - Perform sentiment analysis on review texts to measure customer satisfaction.
  - Do topic modeling to find common themes in positive and negative reviews.


**5. Predictive Analysis**
- **Assigned to:** Julia and Haig
- **Responsibilities:**
  - Train machine learning models to predict review ratings based on review text and product features.
  - Evaluate model performance and improve it as needed.


References:

1. Justifying recommendations using distantly-labeled reviews and fine-grained aspects  
Jianmo Ni, Jiacheng Li, Julian McAuley  
Empirical Methods in Natural Language Processing (EMNLP), 2019
